In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
import openai
from typing import Generator, List, Tuple

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("DEEPSEEK_API_KEY")
openai.base_url = "https://api.deepseek.com"  # Set custom base URL
MODEL_NAME = "deepseek-chat"  # Confirm model name with DeepSeek docs

def format_history(chat_history: List[Tuple[str, str]]) -> List[dict]:
    """Convert Gradio history to API-compatible format"""
    messages = []
    for user_msg, bot_msg in chat_history:
        if bot_msg is not None:
            messages.extend([
                {"role": "user", "content": user_msg},
                {"role": "assistant", "content": bot_msg}
            ])
    return messages

def stream_response(messages: List[dict]) -> Generator[str, None, None]:
    """Stream response using DeepSeek API"""
    try:
        full_response = ""
        stream = openai.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            stream=True,
            temperature=0.7,
        )
        
        for chunk in stream:
            content = chunk.choices[0].delta.content
            if content:
                full_response += content
                yield full_response

    except Exception as e:
        yield f"API Error: {str(e)}"

def chat_response(message: str, history: List[Tuple[str, str]]) -> Generator[List[Tuple[str, str]], None, None]:
    """Handle chat interaction with streaming"""
    base_messages = format_history(history)
    base_messages.append({"role": "user", "content": message})
    
    full_response = ""
    for chunk in stream_response(base_messages):
        full_response = chunk
        yield history + [(message, full_response)]

    formatted_response = format_markdown(full_response)
    yield history + [(message, formatted_response)]

def format_markdown(text: str) -> str:
    """Format response with markdown support"""
    return text.replace("\n", "  \n")  # Preserve line breaks

with gr.Blocks(theme=gr.themes.Soft(), css=".gradio-container {max-width: 800px}") as app:
    gr.Markdown("# DeepSeek Chat Clone")
    
    chatbot = gr.Chatbot(
        elem_id="chatbot",
        height=700,
        show_copy_button=True,
        avatar_images=(None, "/path/to/deepseek-logo.png")
    )
    
    with gr.Row():
        msg = gr.Textbox(
            label="Message",
            placeholder="Ask me anything...",
            container=False,
            autofocus=True,
            scale=7
        )
        submit_btn = gr.Button("Send", variant="primary", scale=1)
    
    clear_btn = gr.Button("Clear History", variant="secondary")

    def user(message: str, history: list) -> Tuple[str, list]:
        return "", history + [(message, None)]

    # Event handlers
    msg.submit(
        user, [msg, chatbot], [msg, chatbot], queue=False
    ).then(
        chat_response, [msg, chatbot], [chatbot]
    )

    submit_btn.click(
        user, [msg, chatbot], [msg, chatbot], queue=False
    ).then(
        chat_response, [msg, chatbot], [chatbot]
    )

    clear_btn.click(
        lambda: [], None, chatbot, queue=False
    )

if __name__ == "__main__":
    app.launch(share=True)  # Always share with auto port selection